In [1]:
# -*- coding: UTF-8 -*-
import os
import requests
from bs4 import BeautifulSoup
import traceback
import string
import math

In [2]:
#主要會改的地方
#在cell6
#url_list = [版名]
#在cell7
#path = r"儲存路徑\PTT"

In [3]:
def get_article_url(path,flag,article_url):
    try:
        cur_flag = flag
        r = requests.get(article_url)
        soup = BeautifulSoup(r.text, "lxml")
        results = soup.select('span.article-meta-value')
        tags = soup.select('span.article-meta-tag')
        presults = soup.select('div.bbs-content')
        tag_list = []
        for i in range(0,len(tags)-1):
            tag_list.append(tags[i].text)
        print(len(results))
        if(len(results) == 4):
            title = maketitle(results)
            cur_flag = int(title[0:title.find("_")])
            content = makecontent(presults[2].text)
            if(len(content)>=100):
                print("title",title)
                titleurl = title + "\n" + article_url + "\n" 
                save_to_file(path,"url.txt",titleurl)
            else:
                print(len(content),"<100","title",title)
        else:
            try:
                up = "作者"
                p = "標題_"+article_url[article_url.find("shoes/")+6:article_url.find(".html")]
                date = "0"
                for i in range(0,len(results)-1):
                    print(i)
                    if(tags[i].text == "時間"):
                        time = results[i].text
                        y=time[-4:len(time)]
                        m=findmonth(time[4:7])
                        d=finddate(time[8:10])
                        date=y+m+d                
                        print("time:",date)
                    elif(tags[i].text == "作者"):
                        up = results[i].text[0:results[i].text.index("(")]
                        print("作者:",up)
                    elif(tags[i].text == "標題"):
                        p = results[i].text.replace("\\","_").replace("|","_").replace(":","_").replace("<","_").replace(">","_").replace(r'*',"_").replace(r'"',"_").replace(r'?',"_").replace(r'\\',"/").replace(r'/',"_")+up+".txt"
                        print("標題:",p)

                content = makecontent(presults[2].text)
                try:
                    if(content.find("時間:")>=1):
                        time=content[content.index("時間")+4:content.index("時間")+28]
                        try:
                            int(time)
                            y=time[-4:len(time)]
                            m=findmonth(time[4:7])
                            d=finddate(time[8:10])
                            date=y+m+d
                            print("time:",date)    
                        except:
                            date = "0"
                        content = content.replace(content[content.index("時間"):content.index("時間")+28],"")
                    if(content.find("標題:")>=1):
                        p=content[content.index("標題")+4:content[content.index("標題"):len(content)-1].index("\n")+1]
                        print("標題:",p)
                        content = content.replace(content[content.index("標題"):content[content.index("標題"):len(content)-1].index("\n")+1],"\n")
                    if(content.find("作者:")>=1):
                        up=content[content.index("作者")+4:content.index("(")]
                        print("作者:",up)
                        content = content.replace(content[content.index("作者"):content.index(")")+1],"\n")
                    title = date + "_" + p + "_" + up + ".txt"
                    cur_flag = int(title[0:title.find("_")])
                    if(len(content)>=100):
                        print("title",title)
                        titleurl = title + "\n" + article_url + "\n" 
                        save_to_file(path,"url",titleurl)
                    else:
                        print(len(content),"<100","title",title)
                except:
                    print('不明的程式中斷')
                    traceback.print_exc()
            except:
                print('不明的程式中斷')
                traceback.print_exc()
    except:
        cur_flag = 0
        print('不明的程式中斷')
        traceback.print_exc()
    return cur_flag

In [ ]:
def save_to_file(path,file_name, contents):
    fh = open(path+"/"+file_name,'a',encoding="utf-8_sig")
    fh.write(contents)
    fh.close()

def findmonth(month):
    if month=="Jan":
        return "01"
    if month=="Feb":
        return "02"
    if month=="Mar":
        return "03"
    if month=="Apr":
        return "04"
    if month=="May":
        return "05"
    if month=="Jun":
        return "06"
    if month=="Jul":
        return "07"
    if month=="Aug":
        return "08"
    if month=="Sep":
        return "09"
    if month=="Oct":
        return "10"
    if month=="Nov":
        return "11"
    if month=="Dec":
        return "12"
    
def finddate(date):
    if date==" 1":
        return "01"
    if date==" 2":
        return "02"
    if date==" 3":
        return "03"
    if date==" 4":
        return "04"
    if date==" 5":
        return "05"
    if date==" 6":
        return "06"
    if date==" 7":
        return "07"
    if date==" 8":
        return "08"
    if date==" 9":
        return "09"
    else:
        return date

def sweeper(al):
    al.remove(al[0])
    revind=[]
    ind=0
    for aa in al:
        if(aa.count("推")>=1 or aa.count("→")>=1 or aa.count("噓")>=1):
            if(aa.count(":")>=1):
                if(aa.count("http")>=1):
                    revind.append(aa)
                else:
                    alist=[]
                    flag =0
                    for i in range(flag,len(aa)-1):
                        if (aa[i] == " "):
                            nc = aa[flag:i]
                            flag=i+1
                            alist.append(nc)
                    item=""
                    for itemindex in range(2,len(alist)-1):
                        item=item+alist[itemindex]+" "
                    al[ind]=item
        if(aa.count("※")>=1 or aa.count("--")>=1 or aa.count("http")>=1 or aa.count("◆")>=1):
            if(aa not in revind):
                revind.append(aa)
        ind=ind+1
    for i in revind:
        if(i in al):
            al.remove(i)
    content = ""
    for aa in al:
        content = content+aa+"\n"
    return content

def findup(target):
    i=target.find(" ")
    up='_'+target[0:i]
    return up

def maketitle(target):
    y=target[3].text[-4:len(target[3].text)]
    m=findmonth(target[3].text[4:7])
    d=finddate(target[3].text[8:10]) 
    try:
        date=y+m+d
        int(date)
        str(date)
    except:
        
        print("ymd",y,m,d)
        date ="0"
    up=findup(target[0].text)
    a=date+"_"+target[2].text.replace("\\","_").replace("|","_").replace(":","_").replace("<","_").replace(">","_").replace(r'*',"_").replace(r'"',"_").replace(r'?',"_").replace(r'\\',"/").replace(r'/',"_")+up+".txt"
    return a

def makecontent(target):
    al = []
    flag = 0
    for i in range(flag, len(target)-1):
        if (target[i] == "\n"):
            row = target[flag:i]
            flag=i+1
            al.append(row)
    al.append(target[flag:len(target)-1])
    content = sweeper(al)
    return content

In [ ]:
def get_all_href(flag,url):
    r = requests.get(url)
    soup = BeautifulSoup(r.text, "lxml")
    results = soup.select("div.title")
    cur_flag = flag
    stop_flag = []
    for item in results:
        a_item = item.select_one("a")
        if a_item:
            a_flag = get_article_url(path,flag,article_url='https://www.ptt.cc'+a_item.get('href'))
            if(a_flag>=cur_flag):
                cur_flag = a_flag
            if(a_flag >= flag):
                print(a_flag, ">=", flag,":",a_flag >= flag)
                stop_flag.append(0)
                print("0")
            else:
                stop_flag.append(1)
                print("1")
            
            print("cur_flag",cur_flag)
    print(stop_flag)
    print('------------------ next page ------------------')
    return stop_flag

In [ ]:
url_list = [
   "shoes"
]

In [ ]:
for url in url_list:
    url="https://www.ptt.cc/bbs/"+url+"/index.html"
    r = requests.get(url)
    soup = BeautifulSoup(r.text, "lxml")
    print('url:', url)
    boardname = url[23:url.find(r"/index")]
    print(boardname)
    path = r"C:\Users\Suisei_Saika\Desktop\PTT"+boardname
    print(path)
    if not os.path.isdir(path):
        os.mkdir(path)
    flag = 0
    f = os.listdir(path)
    stop_flag = get_all_href(flag,url) 
    if(0 in stop_flag and 1 in stop_flag):
        if(stop_flag.index(1)>stop_flag.index(0)):
            btn = soup.select('div.btn-group > a')
            if btn:
                next_page_url = 'https://www.ptt.cc' + btn[3]['href']
                url = next_page_url
                print('url:',url)
                ii=url.index('index')
                hi=url.index('.html')
                pages=url[ii+5:hi]
                stop_flag = get_all_href(flag,url = url) 
                for page in range(1,int(pages)):
                    if(0 in stop_flag and 1 in stop_flag):
                        if(stop_flag.index(1)>stop_flag.index(0)):
                            r = requests.get(url)
                            soup = BeautifulSoup(r.text, "lxml")
                            btn = soup.select('div.btn-group > a')
                            if btn:
                                next_page_url = 'https://www.ptt.cc' + btn[3]['href']
                                url = next_page_url
                                print('url:',url)
                                stop_flag = get_all_href(flag,url = url)
                        else:
                            print("資料為最新")
                            break
                    elif(1 not in stop_flag):
                        r = requests.get(url)
                        soup = BeautifulSoup(r.text, "lxml")
                        btn = soup.select('div.btn-group > a')
                        if btn:
                            next_page_url = 'https://www.ptt.cc' + btn[3]['href']
                            url = next_page_url
                            print('url:',url)
                            stop_flag = get_all_href(flag,url = url)
        else:
            print("資料為最新")
    elif(1 not in stop_flag):
        btn = soup.select('div.btn-group > a')
        if btn:
            next_page_url = 'https://www.ptt.cc' + btn[3]['href']
            url = next_page_url
            print('url:',url)
            ii=url.index('index')
            hi=url.index('.html')
            pages=url[ii+5:hi]
            stop_flag = get_all_href(flag,url = url) 
            for page in range(1,int(pages)):
                if(0 in stop_flag and 1 in stop_flag):
                    if(stop_flag.index(1)>stop_flag.index(0)):
                        r = requests.get(url)
                        soup = BeautifulSoup(r.text, "lxml")
                        btn = soup.select('div.btn-group > a')
                        if btn:
                            next_page_url = 'https://www.ptt.cc' + btn[3]['href']
                            url = next_page_url
                            print('url:',url)
                            stop_flag = get_all_href(flag,url = url)
                    else:
                        print("資料為最新")
                        break
                elif(1 not in stop_flag):
                    r = requests.get(url)
                    soup = BeautifulSoup(r.text, "lxml")
                    btn = soup.select('div.btn-group > a')
                    if btn:
                        next_page_url = 'https://www.ptt.cc' + btn[3]['href']
                        url = next_page_url
                        print('url:',url)
                        stop_flag = get_all_href(flag,url = url)

In [ ]:
# url='https://www.ptt.cc/bbs/shoes/index921.html'
# r = requests.get(url)
# soup = BeautifulSoup(r.text, "lxml")
# print('url:', url)
# get_all_href(url) 
# btn = soup.select('div.btn-group > a')
# if btn:
#     next_page_url = 'https://www.ptt.cc' + btn[3]['href']
#     url = next_page_url
#     print('url:',url)
#     ii=url.index('index')
#     hi=url.index('.html')
#     pages=url[ii+5:hi]
#     get_all_href(url = url) 
# for page in range(1,int(pages)):
#     r = requests.get(url)
#     soup = BeautifulSoup(r.text, "lxml")
#     btn = soup.select('div.btn-group > a')
#     if btn:
#         next_page_url = 'https://www.ptt.cc' + btn[3]['href']
#         url = next_page_url
#         print('url:',url)
#         get_all_href(url = url)